# Credit Scoring Model Using Scikit-Learn

### Credit Scoring
A credit score is a numerical expression based on a level analysis of a person's credit files, to represent the creditworthiness of an individual. Traditionally, a credit score was primarily based on credit report information typically sourced from credit bureaus. However, with the proliferation of data science, institutions of any size can develop their own credit scoring system and sharpen them for applications to their target markets

### Scikit-learn
Scikit-learn  is an Open source machine learning library Python, and features various classification, regression and clustering algorithms, and is designed to interoperate with the Python numerical and scientific libraries NumPy and SciPy.

Scikit-learn prides itself on the following aspects
- Simple and efficient tools for data mining and data analysis
- Accessible to everybody, and reusable in various contexts
- Built on NumPy, SciPy, and matplotlib
- Open source, commercially usable - BSD license


### Lending Club Dataset

Lending Club is a US peer-to-peer lending company, headquartered in San Francisco, California. Lending Club is the world's largest peer-to-peer lending platform. The company states that $15.98 billion in loans had been originated through its platform up to 31 December 2015.

We shall use their loan data for this exercise. Their public datasets can be accessed from https://www.lendingclub.com/info/download-data.action

## Let’s Get Started

In [1]:
import pandas as pd, numpy as np
import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.model_selection import GridSearchCV

import sklearn
from sklearn_pandas import DataFrameMapper

import matplotlib.pyplot as plt

### Step1
Let’s first load the loans data. For your conviniebce, we have made this available as exp-lending-club-data.csv on rorocloud. We shall use a pandas dataframe to read this data into, as pandas is an excellent tool for data preparation and analysis.

In [2]:
IN_DATAFILE='https://s3.amazonaws.com/rorodata-datasets/lending-club-data.csv'
loans = pd.read_csv(IN_DATAFILE, infer_datetime_format = True)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Step 2:  
The dataframe loans has 68 columns of data. However, we shall work with only those columns that we find relevant for our model. We have picked 18 columns for this exercise, these have been identified in the list named features. We want to use these features to predict the column bad_loan (1 for bad loan, 0 for otherwise)- hence, this is a binary classification problem
To create this data subset in pandas is a cinch
> clean_data=loans[features+[response]].dropna()

In [3]:
features = ['grade',                     # grade of the loan (categorical)
            'sub_grade_num',             # sub-grade of the loan as a number from 0 to 1
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'payment_inc_ratio',         # ratio of the monthly payment to income
            'delinq_2yrs',               # number of delinquincies 
            'delinq_2yrs_zero',          # no delinquincies in last 2 years
            'inq_last_6mths',            # number of creditor inquiries in last 6 months
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'open_acc',                  # number of open credit accounts
            'pub_rec',                   # number of derogatory public records
            'pub_rec_zero',              # no derogatory public records
            'revol_util',                # percent of available credit being used
           ]
response='bad_loans'

In [4]:
clean_data=loans[features+[response]].dropna()

### Step 3:  
Now that we have the data set that we need for modelling, we need to apply some preprocessing steps to get the data into a form that scikit-learn algorithms can use. In almost all cases, scikit-learn algorithms have the interface model.fit(X,y), where X and y are numerical matrices representing the features and the target respectively.
We will do only one simple preprocessing steps for this tutorial, i.e. convert categorical data into one-hot encoded format. We use the scikit proprocessing model called LabelBinarizer() on only the Categorial columns.
At the end of this step, we have a dataset (X,y)  where X and y are numpy arrays  and all columns are numerical values (with categorical variables being one-hot encoded)

In [5]:
numerical_cols=['sub_grade_num', 'short_emp', 'emp_length_num','dti', 'payment_inc_ratio', 'delinq_2yrs', \
                'delinq_2yrs_zero', 'inq_last_6mths', 'last_delinq_none', 'last_major_derog_none', 'open_acc',\
                'pub_rec', 'pub_rec_zero','revol_util']

categorical_cols=['grade', 'home_ownership', 'purpose']

mapper = DataFrameMapper([
('grade',sklearn.preprocessing.LabelBinarizer()),
('home_ownership', sklearn.preprocessing.LabelBinarizer()),
('purpose', sklearn.preprocessing.LabelBinarizer()),
        
    ])

X1=mapper.fit_transform(clean_data)


X2=np.array(clean_data[numerical_cols])


X = np.hstack((X1,X2)) #Combines X1 and X2 side by side, i.e. stacks them horizontally
y=np.array(clean_data['bad_loans'])



### Step 4
The first step in build and testing models is to do a split of train-test data, so that we have unbiased estimates of the model’s error. We keep aside a third of the data for testing purposes, and use 2/3rds for training. This is accomplished very simply in Scikit-learn as follows

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=100, stratify=y)

### Step 5
The actual model building in scikit is tremendously simplified, and follows the same pattern as below


```
model=scikit_model_class()       initializes  a model of class scikit_model_class
model.fit(X_train, y_train)      fits a model on the training data set
model.score(X_test, y_test)      evaluates the performance of the model on the test data set
```

#### Fit a Logistic Regression model

In [6]:
log_lm = LogisticRegression()
log_lm.fit(X_train, y_train)
log_lm.score(X_test, y_test)

0.81184643148500657

#### Fit a Gradient Boosting model

In [7]:
grd = GradientBoostingClassifier(n_estimators=100)
grd.fit(X_train, y_train)
grd.score(X_test, y_test)

0.81142616993399419

#### Fit a Decision Tree model

In [8]:
dtree=DecisionTreeClassifier()
parameters = {'max_depth':[5, 10, 15, 20, 25, 32]}
dtree_gs = GridSearchCV(dtree, parameters, cv=5)
dtree_gs.fit(X_train, y_train)
print(dtree_gs.score(X_test, y_test))

0.810511483029


#### Fit a Random Tree model

In [9]:
rf=RandomForestClassifier()
parameters = {'max_depth':[5, 15], 'n_estimators':[10,30]}
rf_gs = GridSearchCV(rf, parameters)
rf_gs.fit(X_train, y_train)
print(rf_gs.score(X_test, y_test))

0.811154235989


### Step 5: 
To score new loan applications, we must remember to do the following things
1.	Apply the same data columns as we have used in the modelling process
2.	Apply the same pre-processing steps, including the same Label Binarizers that were trained on the original training data (i.e. do not refit )
3.	Apply the models that were trained on the training dataset (again, do not refit) 


In [10]:
# Let's try it out on this sample application
a={ 'delinq_2yrs': 0.0,
 'delinq_2yrs_zero': 1.0,
 'dti': 8.7200000000000006,
 'emp_length_num': 0,
 'grade': 'F',
 'home_ownership': 'RENT',
 'inq_last_6mths': 3.0,
 'last_delinq_none': 1,
 'last_major_derog_none': 1,
 'open_acc': 2.0,
 'payment_inc_ratio': 4.5,
 'pub_rec': 0.0,
 'pub_rec_zero': 1.0,
 'purpose': 'vacation',
 'revol_util': 98.5,
 'short_emp': 0,
 'sub_grade_num': 1.0}

In [11]:
def preProcess(a):
    data=list(a.values())
    colz=list(a.keys())
    dfx=pd.DataFrame(data=[data], columns=colz)
    dfx

    XX1=mapper.transform(dfx)
    XX2=dfx[numerical_cols]
    XX = np.hstack((XX1,XX2))
    return XX

In [13]:
XX=preProcess(a)


0.48829986303342249

In [18]:
print(" Accrording to the Logistic Regression model,the probability of loan default is ", log_lm.predict_proba(XX)[:,1][0])

 Accrording to the Logistic Regression model,the probability of loan default is  0.488299863033


In [20]:
print(" Accrording to the Logistic Regression model,the probability of loan default is ", grd.predict_proba(XX)[:,1][0])

 Accrording to the Logistic Regression model,the probability of loan default is  0.534399174411
